In [1]:
import cPickle
import os.path

api_key = cPickle.load( file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'r' ) )

In [2]:
import cPickle
import os.path

cPickle.dump( api_key, file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'wb' ) )

In [3]:
import mediacloud, json

mc = mediacloud.api.AdminMediaCloud( api_key)


In [4]:
def corenlp_annotated( s ):
    corenlp = s['corenlp']
    
    if 'annotated' not in corenlp:
        return True
    
    if corenlp['annotated'] == 'false':
        return False
    
    return True

In [5]:
#import cliff
from mediameter.cliff import Cliff

cliff_api = Cliff('http://civicdev.media.mit.edu', 8080)

# Flags

In [6]:
reload_stories = False
reprocess_with_cliff = True


In [7]:
import shelve

In [8]:
controversy_stories = []

fetch_size = 100

if reload_stories:
    
    last_processed_stories_id = 0

    controversy_stories_shelve = shelve.open((os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_shelve.shelve' ) ))
    
    while True:
        print "fetching with last_processed_stories_id", last_processed_stories_id
        stories = mc.storyList( solr_query='{~ controversy:1113 }', rows=fetch_size, corenlp=True, 
                               last_processed_stories_id=last_processed_stories_id, sentences=True, text=True)

        if len( stories ) == 0:
            break
            
        for s in stories:          
            
            #del s['description']
            
            controversy_stories_shelve[ str(s[ 'stories_id']) ] = s
            
        last_processed_stories_id = stories[-1]['processed_stories_id']
        
        #if len( controversy_stories_shelve.keys()) > 200:
        #    break

    print len ( controversy_stories_shelve.keys())
    controversy_stories_shelve.close()


In [15]:
if reprocess_with_cliff:
    
    controversy_stories_shelve  = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_shelve.shelve' ), flag='r' )
    controversy_stories_corenlp = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_corenlp.shelve' ) , flag='n' )

    stories_ids = sorted( controversy_stories_shelve.keys() )

    non_corenlp_annotated_stories = []
    cliff_error_stories = []
    cliff_no_result_stories = []
    cliff_no_people_stories = []

    for stories_id in stories_ids:
        s = controversy_stories_shelve[ stories_id]

        cliff_info = {}

        if corenlp_annotated( s ):
            try:
                cliff_info['cliff_results'] = cliff_api.parseNlpJson( s['corenlp'])
            except:
                print 'Cliff Error on ', stories_id

                cliff_error_stories.append( stories_id )

                continue

            if 'results' not in cliff_info['cliff_results']:
                cliff_no_result_stories.append( stories_id )

            else:

                results = cliff_info['cliff_results']['results']

                if 'people' in results:
                    people = results['people']
                    names = [ p['name'] for p in people ]

                    cliff_info[ 'names'] = names

                else:
                    cliff_no_people_stories.append( stories_id )

            controversy_stories_corenlp[ stories_id ] = cliff_info

        else:  # not corenlp_annotated

            print stories_id, 'not annotated'

            non_corenlp_annotated_stories.append( stories_id )

    cliff_problems = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/cliff_problmes.shelve' ) , flag='n' )

    cliff_problems[ 'cliff_error_stories'] = cliff_error_stories
    cliff_problems[ 'cliff_no_result_stories'] = cliff_no_result_stories
    cliff_problems[ 'cliff_no_people_stories'] = cliff_no_people_stories
    cliff_problems[ 'non_corenlp_annotated_stories'] = non_corenlp_annotated_stories
    cliff_problems.close()
    controversy_stories_corenlp.close()
    controversy_stories_shelve.close()

124099751 not annotated
164918000 not annotated
169464344 not annotated
174255563 not annotated
217751583 not annotated
219285013 not annotated
229263527 not annotated
237628466 not annotated
238668953 not annotated
239207216 not annotated
241748601 not annotated
24238936 not annotated
242912598 not annotated
Cliff Error on  244729537
244742084 not annotated
245244212 not annotated
245354910 not annotated
Cliff Error on  248559106
248628727 not annotated
Cliff Error on  249174413
250789636 not annotated
250790429 not annotated
250795713 not annotated
254530873 not annotated
Cliff Error on  257513620
Cliff Error on  257549636
Cliff Error on  258264523
Cliff Error on  258993578
Cliff Error on  259438933
259719237 not annotated
Cliff Error on  260567305
Cliff Error on  260783901
Cliff Error on  260785929
Cliff Error on  260837029
Cliff Error on  260838223
Cliff Error on  260901237
Cliff Error on  260924925
Cliff Error on  260938808
Cliff Error on  260943694
Cliff Error on  260953097
Cliff

In [17]:
controversy_stories_shelve  = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_shelve.shelve' ), flag='r' )
controversy_stories_corenlp = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_corenlp.shelve' ) , flag='r' )
cliff_problems = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/cliff_problmes.shelve' ) , flag='r' )

print len ( controversy_stories_shelve.keys()), 'total stories'
print len (controversy_stories_corenlp.keys() ), 'controvery_stories corenlp annotated'
print "\t", len ( cliff_problems['cliff_error_stories']), 'cliff error stories'
print "\t", len ( cliff_problems['cliff_no_result_stories']), 'cliff no result stories'
print "\t", len ( cliff_problems['cliff_no_people_stories']), 'cliff no people stories'
print len ( cliff_problems['non_corenlp_annotated_stories'] ), 'non corenlp annotated stories'



16048 total stories
14850 controvery_stories corenlp annotated
	376 cliff error stories
	1 cliff no result stories
	0 cliff no people stories
822 non corenlp annotated stories


In [10]:
len(controversy_stories)

13800

In [ ]:
import itertools

edge_counts = Counter()

for names in stories_name_map.values():
    edges = list (itertools.combinations( names, 2 ) )
    
    for edge in edges:
        if edge not in edges:
            edge_counts[edge] = 0
    
        edge_counts[ edge ] += 1

edge_counts.most_common()    

In [ ]:
from collections import Counter

c = Counter (  dict( [ ( name, len ( names_stories_map[ name ] ) ) for name in names_stories_map.keys() ]  ) )
             

del c['Sepp Blatter']
del c['Ray Rice']

c.most_common()
